In [9]:
import os
from pydantic_ai.settings import ModelSettings

BASE_URL = f"http://localhost:8000/v1"

os.environ["BASE_URL"]    = BASE_URL
os.environ["OPENAI_API_KEY"] = "abc-123"   

print("Config set:", BASE_URL)

import os
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

provider = OpenAIProvider(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
)

agent_model = model = OpenAIModel("Qwen3-30B-A3B", provider=provider, settings=ModelSettings(temperature=0))

Config set: http://localhost:8000/v1


In [10]:
from flask import Flask, request, jsonify
from threading import Thread
import dateutil
import json

from src.agents.meeting_agent import meeting_agent
from src.agents.priority_agent import get_priority_async
from src.agents.slots_agent import slots_agents

from src.utils.attendees import get_attendees_with_from
from src.utils.interval_handling import get_free_time_slots,get_free_time_slots_priority, find_earliest_meeting_slot

In [11]:
app = Flask(__name__)
received_data = []

In [12]:
import time
import asyncio
from datetime import datetime, timedelta
from dateutil import parser

# Alternative version with even more optimization
async def your_meeting_assistant_ultra_optimized(data): 
    try:
        # Start timer for all concurrent operations
        start_total = time.time()
    
        # Start timer
        start = time.time()
        
        # Create tasks for all independent operations
        priority_task = get_priority_async(data["EmailContent"])
        meeting_agent_task = meeting_agent.run(data["EmailContent"])
    
        # End timer
        end = time.time()
        
        # Calculate and print runtime
        runtime = end - start
        print(f"meeting_agent Runtime: {runtime:.4f} seconds")
        
    
        # Start timer
        start = time.time()
        
        # Get attendees synchronously (assuming this is fast)
        attendees = get_attendees_with_from(data)
        print("Extracted Attendees")
        print(attendees)
    
        # End timer
        end = time.time()
        
        # Calculate and print runtime
        runtime = end - start
        print(f"get_attendees_with_from Runtime: {runtime:.4f} seconds")
        
        # Wait for priority and meeting agent results
        priority_result, meeting_agent_results = await asyncio.gather(
            priority_task,
            meeting_agent_task
        )
        
        # Process results
        priority = priority_result.data.priority
        data["Priority"] = priority
        print("pri",priority)
        
        response_data_meeting = {
            "start_timeframe": meeting_agent_results.data.start_timeframe,
            "end_timeframe": meeting_agent_results.data.end_timeframe,
            "duration_minutes": meeting_agent_results.data.duration_minutes,
            "success": True,
            "message": "Successfully extracted meeting timeframes"
        }
        
        print("EXTRACTED TIME ELEMENTS")
        print(response_data_meeting)
        
        # Get free slots
        free_slots, user_schedule_dict = get_free_time_slots(
            attendees, 
            response_data_meeting["duration_minutes"], 
            response_data_meeting["start_timeframe"], 
            response_data_meeting["end_timeframe"],
            slots_agents
        )
        
        print("Extracted Free Slots")
        if free_slots == 'No available time slots found for the specified duration\n ':
            print('No available time slots found for the specified duration\n ')
            free_slots = get_free_time_slots_priority(
                attendees, 
                response_data_meeting["duration_minutes"], 
                response_data_meeting["start_timeframe"], 
                response_data_meeting["end_timeframe"], 
                priority, 
                len(data["Attendees"])
            )
        print(free_slots)
    
        response_data_slots = find_earliest_meeting_slot(free_slots, int(response_data_meeting["duration_minutes"]), metadata="")
        
        print("Suggested Slot")
        print(response_data_slots)
        
        print("After Processing\n")
        # Parse the start time string to datetime object
        start_time_dt = dateutil.parser.isoparse(response_data_slots["start_time_of_meeting"])
        
        # Add the duration
        end_time_dt = start_time_dt + timedelta(minutes=int(response_data_slots["duration_minutes"]))
        
        # Convert back to ISO format string
        end_time_str = end_time_dt.isoformat()
    
        response_data_slots['end_time_of_meeting'] = str(end_time_str)
    
        print(response_data_slots)
    
        # Parse the response
        raw = response_data_slots["start_time_of_meeting"].strip()
        try:
            dt = parser.isoparse(raw)
        except Exception:
            dt = parser.parse(raw) 
        
        available_start_time = dt.strftime("%Y-%m-%dT%H:%M:%S%z")
        available_end_dt = dt + timedelta(minutes=response_data_meeting["duration_minutes"])
        available_end_time = available_end_dt.strftime("%Y-%m-%dT%H:%M:%S%z")
    
        # End total timer
        end_total = time.time()
        total_runtime = end_total - start_total
        print(f"Total optimized runtime: {total_runtime:.4f} seconds")
    
        # Build output efficiently
        output = {
            "Request_id": data["Request_id"],
            "Datetime": data["Datetime"],
            "Location": data["Location"],
            "From": data["From"],
            "Attendees": [
                {**attendee, "events": user_schedule_dict[attendee["email"]]} 
                for attendee in data["Attendees"]
            ],
            "Subject": data["Subject"],
            "EmailContent": data["EmailContent"],
            "EventStart": available_start_time,
            "EventEnd": available_end_time,
            "Duration_mins": str(response_data_meeting["duration_minutes"]),
            "MetaData": response_data_slots['metadata']
        }
        
        return output
    except:
        start_time = datetime.fromisoformat(response_data_meeting['start_timeframe'])
        end_time = start_time + timedelta(minutes=response_data_meeting['duration_minutes'])
        output = {
            "Request_id": data["Request_id"],
            "Datetime": data["Datetime"],
            "Location": data["Location"],
            "From": data["From"],
            "Attendees": [
                {**attendee, "events": user_schedule_dict[attendee["email"]]} 
                for attendee in data["Attendees"]
            ],
            "Subject": data["Subject"],
            "EmailContent": data["EmailContent"],
            "EventStart": start_time,
            "EventEnd": end_time,
            "Duration_mins": str(response_data_meeting["duration_minutes"]),
            "MetaData": response_data_slots['metadata']
        }
        
        return output
        
        

In [13]:
#t1
print("test 1")
data = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}
await your_meeting_assistant_ultra_optimized(data)
#t2

print("test 2")
data = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033b5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Client Validation - Urgent",
    "EmailContent": "Hi Team. We’ve just received quick feedback from the client indicating that the instructions we provided aren’t working on their end. Let’s prioritize resolving this promptly. Let’s meet Monday at 9:00 AM to discuss and resolve this issue."
}
await your_meeting_assistant_ultra_optimized(data)

#t3

print("test 3")
data = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033c5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Project Status",
    "EmailContent": "Hi Team. Let's meet on Tuesday at 11:00 A.M and discuss about our on-going Projects."
}
await your_meeting_assistant_ultra_optimized(data)

#t4

print("test 4")
data = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033d5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Client Feedback",
    "EmailContent": "Hi Team. We’ve received the final feedback from the client. Let’s review it together and plan next steps. Let's meet on Wednesday at 10:00 A.M."
}
await your_meeting_assistant_ultra_optimized(data)

test 1
meeting_agent Runtime: 0.0000 seconds
Extracted Attendees
['userone.amd@gmail.com', 'usertwo.amd@gmail.com', 'userthree.amd@gmail.com']
get_attendees_with_from Runtime: 0.0000 seconds


/tmp/ipykernel_21997/814256239.py:48: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  priority = priority_result.data.priority
/tmp/ipykernel_21997/814256239.py:53: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "start_timeframe": meeting_agent_results.data.start_timeframe,
/tmp/ipykernel_21997/814256239.py:54: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "end_timeframe": meeting_agent_results.data.end_timeframe,
/tmp/ipykernel_21997/814256239.py:55: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "duration_minutes": meeting_agent_results.data.duration_minutes,


pri mid
EXTRACTED TIME ELEMENTS
{'start_timeframe': '2025-07-24T00:00:00+05:30', 'end_timeframe': '2025-07-24T23:59:59+05:30', 'duration_minutes': 30, 'success': True, 'message': 'Successfully extracted meeting timeframes'}
[{'StartTime': '2025-07-23T18:00:00+05:30', 'EndTime': '2025-07-24T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}, {'StartTime': '2025-07-24T10:30:00+05:30', 'EndTime': '2025-07-24T11:00:00+05:30', 'NumAttendees': 3, 'Attendees': ['userthree.amd@gmail.com', 'usertwo.amd@gmail.com', 'userone.amd@gmail.com'], 'Summary': 'Agentic AI Project Status Update'}, {'StartTime': '2025-07-24T18:00:00+05:30', 'EndTime': '2025-07-25T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}]
[{'StartTime': '2025-07-23T18:00:00+05:30', 'EndTime': '2025-07-24T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}, {'StartTime': '2025-07-24T10:30:00+05:30', 'EndTime': '2025-07-24T11:00:00+05:30',

{'Request_id': '6118b54f-907b-4451-8d48-dd13d76033d5',
 'Datetime': '19-07-2025T12:34:55',
 'Location': 'IISc Bangalore',
 'From': 'userone.amd@gmail.com',
 'Attendees': [{'email': 'usertwo.amd@gmail.com',
   'events': [{'StartTime': '2025-07-23T10:00:00+05:30',
     'EndTime': '2025-07-23T11:00:00+05:30',
     'NumAttendees': 1,
     'Attendees': ['SELF'],
     'Summary': 'Client Meeting - IMPORTANT'},
    {'StartTime': '2025-07-23T10:30:00+05:30',
     'EndTime': '2025-07-23T11:30:00+05:30',
     'NumAttendees': 3,
     'Attendees': ['userthree.amd@gmail.com',
      'usertwo.amd@gmail.com',
      'userone.amd@gmail.com'],
     'Summary': 'Client Feedback'},
    {'StartTime': '2025-07-23T18:00:00+05:30',
     'EndTime': '2025-07-24T09:00:00+05:30',
     'NumAttendees': 1,
     'Attendees': ['SELF'],
     'Summary': 'Off Hours'}]},
  {'email': 'userthree.amd@gmail.com',
   'events': [{'StartTime': '2025-07-23T10:30:00+05:30',
     'EndTime': '2025-07-23T11:30:00+05:30',
     'NumAttend

In [14]:
from flask import Flask, request, jsonify
import asyncio
import json

app = Flask(__name__)
received_data = []

@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    print(f"\n Received: {json.dumps(data, indent=2)}")
    
    # Run async function in sync context
    new_data = asyncio.run(your_meeting_assistant_ultra_optimized(data))

    received_data.append(data)
    print(f"\n\n\n Sending:\n {json.dumps(new_data, indent=2)}")
    return jsonify(new_data)

def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://129.212.191.103:5000
Press CTRL+C to quit



 Received: {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033d5",
  "Datetime": "19-07-2025T12:34:55",
  "Location": "IISc Bangalore",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com"
    },
    {
      "email": "userthree.amd@gmail.com"
    }
  ],
  "Subject": "Client Feedback",
  "EmailContent": "Hi Team. We\u2019ve received the final feedback from the client. Let\u2019s review it together and plan next steps. Let's meet on Wednesday at 10:00 A.M."
}
meeting_agent Runtime: 0.0000 seconds
Extracted Attendees
['userone.amd@gmail.com', 'usertwo.amd@gmail.com', 'userthree.amd@gmail.com']
get_attendees_with_from Runtime: 0.0000 seconds


/tmp/ipykernel_21997/814256239.py:48: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  priority = priority_result.data.priority
/tmp/ipykernel_21997/814256239.py:53: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "start_timeframe": meeting_agent_results.data.start_timeframe,
/tmp/ipykernel_21997/814256239.py:54: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "end_timeframe": meeting_agent_results.data.end_timeframe,
/tmp/ipykernel_21997/814256239.py:55: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  "duration_minutes": meeting_agent_results.data.duration_minutes,


pri high
EXTRACTED TIME ELEMENTS
{'start_timeframe': '2025-07-23T10:00:00+05:30', 'end_timeframe': '2025-07-23T23:59:59+05:30', 'duration_minutes': 60, 'success': True, 'message': 'Successfully extracted meeting timeframes'}
[{'StartTime': '2025-07-23T10:30:00+05:30', 'EndTime': '2025-07-23T11:30:00+05:30', 'NumAttendees': 3, 'Attendees': ['userthree.amd@gmail.com', 'usertwo.amd@gmail.com', 'userone.amd@gmail.com'], 'Summary': 'Client Feedback'}, {'StartTime': '2025-07-23T18:00:00+05:30', 'EndTime': '2025-07-24T09:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}]
[{'StartTime': '2025-07-23T10:00:00+05:30', 'EndTime': '2025-07-23T11:00:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Client Meeting - IMPORTANT'}, {'StartTime': '2025-07-23T10:30:00+05:30', 'EndTime': '2025-07-23T11:30:00+05:30', 'NumAttendees': 3, 'Attendees': ['userthree.amd@gmail.com', 'usertwo.amd@gmail.com', 'userone.amd@gmail.com'], 'Summary': 'Client Feedback'}, {'Star

152.57.113.158 - - [20/Jul/2025 07:50:04] "POST /receive HTTP/1.1" 200 -


[{'StartTime': '2025-07-23T10:30:00+05:30', 'EndTime': '2025-07-23T11:30:00+05:30', 'NumAttendees': 3, 'Attendees': ['userthree.amd@gmail.com', 'usertwo.amd@gmail.com', 'userone.amd@gmail.com'], 'Summary': 'Client Feedback'}, {'StartTime': '2025-07-23T16:00:00+05:30', 'EndTime': '2025-07-24T07:30:00+05:30', 'NumAttendees': 1, 'Attendees': ['SELF'], 'Summary': 'Off Hours'}]
Extracted Free Slots
slot 1 : 2025-07-23T11:30:00+05:30 - 2025-07-23T16:00:00+05:30
 
mm [('2025-07-23T11:30:00+05:30', '2025-07-23T16:00:00+05:30')]
Suggested Slot
{'start_time_of_meeting': '2025-07-23T11:30:00+05:30', 'end_time_of_meeting': '2025-07-23T12:30:00+05:30', 'duration_minutes': '60', 'metadata': ''}
After Processing

{'start_time_of_meeting': '2025-07-23T11:30:00+05:30', 'end_time_of_meeting': '2025-07-23T12:30:00+05:30', 'duration_minutes': '60', 'metadata': ''}
Total optimized runtime: 3.6457 seconds



 Sending:
 {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033d5",
  "Datetime": "19-07-2025T12:34